In [ ]:
!time ./setup.sh

In [ ]:
!source activate tensorflow

In [ ]:
!mkdir checkpoint

In [ ]:
# make data smaller
import os
import random
from glob import glob

for filename in glob('data/train2014/*'):
    if random.random() > 0.025: # use only 2.5% of the data
        os.remove(filename)

In [ ]:
# learn art style: train model
# upload steampunk image
!time python style.py --style ../steampunk.jpg --checkpoint-dir checkpoint --epochs 2 --checkpoint-iterations 10 --batch-size 16
#!nvidia-smi # read "Disp.A Memory-Usage: and "GPU Memory Usage" for how much GPU RAM usage; "Volatile GPU-Util" grossly inaccurate
!pip install glances # glances is way better than nvidia-smi in terms of accuracy and UI
!pip install nvidia-ml-py3
!glances
# remove test-dir and other parameters for now

# use small batches if training pictures are bigger



# on 2.5% of pictures and 16 minibatch:
# real    9m59.879s
# user    9m5.264s
# sys     0m48.336s

In [ ]:
!mkdir test_dir
!time python style.py --style ../steampunk.jpg --checkpoint-dir checkpoint --epochs 2 --checkpoint-iterations 10 --batch-size 16 --test examples/style/rain_princess.jpg --test-dir test_dir

## complete runtime here
## appears that it will regress to black; maybe GPU RAM constrained?
## also uses CPU by default, I changed it to be GPU, don't know if RAM is constrained. Have to make this argparse

# every iteration causes checkpoint save (GPU not running) and inference on test image, so use high iteration number

In [ ]:
# apply style on another picture, compare CPU vs GPU
# I don't think there are any other important parameters
!time python evaluate.py --checkpoint checkpoint --in-path examples/style/rain_princess.jpg --out-path ../output_image.jpg

# about the same between GPU (slightly faster) and CPU
# real    0m5.287s
# user    0m3.556s
# sys     0m2.068s

In [ ]:
# apply style to video
# upload video: play_dead.mp4
# use smaller minibatch if video very high resolution
!time python transform_video.py --in-path ../play_dead.mp4   --checkpoint checkpoint   --out-path ../output_video.mp4   --device /gpu:0   --batch-size 16

# GPU
# real    0m19.682s
# user    0m22.548s
# sys     0m6.408s

# CPU
# real    5m50.725s
# user    21m21.868s
# sys     1m17.100s

In [3]:
# add the first part to GPU to reduce runtime by 22 seconds

# Speed
## Try:  more GPUs
## Done: bigger minibatch, less pictures

# Quality:
## Try: increase style weight, increase learning rate
## Done: add test folder

# Notes:
## notice with more iterations, then color of original image more visible; I assume that means the content loss is being reduced
## GPU RAM is full on this EC2 instance by loading up model. Thank goodness it fits into memory! For more powerful GPU, can probably increase minibatch for evaluation
## Currently GPU RAM constrained; not sure how it's evaluating testing picture without sufficient GPU RAM

In [ ]:
--style, --checkpoint-dir # explicitly required
--train-path, --vgg-path # implicitly required

--epochs, --checkpoint-iterations, --batch-size, # optional
--test, --test-dir, --content-weight, --style-weight, --tv-weight, --learning-rate # more optional

In [ ]:
from moviepy.video.VideoClip import VideoClip

clip = VideoFileClip('play_dead.mp4')
cropped = clip.crop(x1=0, y1=0, x2=100, y2=200)
cropped.write_videofile('cropped_video.mp4') # what are dimensions? consider amount of time

# can do the same with ffmpeg and openCV